## Regressione lineare

Consideriamo un sistema $X\beta = y$ dove $X$ ha più righe che colonne. 
Questo accade quando abbiamo più campioni che variabili. Il nostro obiettivo è trovare $\hat{\beta}$ che minimizzi:
$$\big\vert\big\vert X\beta - y \big\vert\big\vert_2$$

In [1]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

data = datasets.load_diabetes()

In [2]:
feature_names=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

iniziamo a trovare una soluzione con sklearn

In [3]:
regr = linear_model.LinearRegression()
%timeit regr.fit(trn, y_trn)

686 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
pred = regr.predict(test)

valutiamo la metrica della soluzione

In [5]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

In [6]:
regr_metrics(y_test, regr.predict(test))

(51.05511944046453, 40.29588998594346)

# Features polinomiali

La regressione lineare calcola i coefficienti $\beta_i$ per:

$$x_0\beta_0 + x_1\beta_1 + x_2\beta_2 = y$$

Aggiungere delle features polinomiali resta ancora un problema di regressione lineare ma con più termini:

$$x_0\beta_0 + x_1\beta_1 + x_2\beta_2 + x_0^2\beta_3 + x_0 x_1\beta_4 + x_0 x_2\beta_5 + x_1^2\beta_6 + x_1 x_2\beta_7 + x_2^2\beta_8 = y$$


Utilizziamo la matrice originale $X$ per calcolare le features aggiuntive:


In [7]:
trn.shape

(353, 10)

Aggiungiamo ora le features polinomiali

In [9]:
poly = PolynomialFeatures(include_bias=False)
trn_feat = poly.fit_transform(trn)

In [10]:
', '.join(poly.get_feature_names(feature_names))

'age, sex, bmi, bp, s1, s2, s3, s4, s5, s6, age^2, age sex, age bmi, age bp, age s1, age s2, age s3, age s4, age s5, age s6, sex^2, sex bmi, sex bp, sex s1, sex s2, sex s3, sex s4, sex s5, sex s6, bmi^2, bmi bp, bmi s1, bmi s2, bmi s3, bmi s4, bmi s5, bmi s6, bp^2, bp s1, bp s2, bp s3, bp s4, bp s5, bp s6, s1^2, s1 s2, s1 s3, s1 s4, s1 s5, s1 s6, s2^2, s2 s3, s2 s4, s2 s5, s2 s6, s3^2, s3 s4, s3 s5, s3 s6, s4^2, s4 s5, s4 s6, s5^2, s5 s6, s6^2'

In [11]:
trn_feat.shape

(353, 65)

In [12]:
regr.fit(trn_feat, y_trn)

LinearRegression()

In [13]:
regr_metrics(y_test, regr.predict(poly.fit_transform(test)))

(58.2390905798078, 45.1230598096478)

In [14]:
%timeit poly.fit_transform(trn)

315 µs ± 35.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Implementazione della generazione delle features

Per l'implementazione della generazione delle features andremo ad usare [numba](http://numba.pydata.org/numba-doc/0.12.2/tutorial_firststeps.html) che si occupa di andare a creare del codice c partendo dal nostro codice python in maniera efficiente.



In [15]:
import numba
print(numba.__version__)

0.54.1


**numba** è un compilatore

### espertimenti con vettorizzazione e codice nativo

creiamo come primo esperimento la generazione di features  

In [16]:
%matplotlib inline

In [18]:
import math, numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

In [19]:
from numba import jit, vectorize, guvectorize, cuda, float32, void, float64

Vedremo l'impatto di:


* evitare di allocare e copiare i dati
* allocazione della memoria efficiente
* vettorizzazione

Se usiamo **numpy** su interi array alla volta, creiamo un sacco di variabili temporanee, e non usiamo la cache.
Se usassimo **numba** per iterare un elemento alla volta non dovremmo allocare grandi array temporanei, possiamo riutilizzare i dati in cache.

Creiamo una funzione non ottimizzata

In [20]:
# Untype and Unvectorized
def proc_python(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [21]:
nobs = 10000
x = np.random.randn(nobs).astype('float32')
y = np.random.randn(nobs).astype('float32')

In [22]:
%timeit proc_python(x,y)   # Untyped and unvectorized

86.3 ms ± 7.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Numpy

Eseguiamo lo stesso calcolo con numpy 

In [23]:
# Typed and Vectorized
def proc_numpy(x,y):
    z = np.zeros(nobs, dtype='float32')
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    z = z * ( z - .88 ) 
    return z

In [24]:
%timeit proc_numpy(x,y)    # Typed and vectorized

38.4 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Vediamo un netto miglioramento nei tempi di calcolo

In [25]:
np.allclose( proc_numpy(x,y), proc_python(x,y), atol=1e-4 )

True

### Numba

Numba offre alcuni decoratori. Ne useremo due differenti:

* @jit: molto generale
* @vectorize: non serve scrivere particolari loop. molto utile quando si sta lavorando con vettori della stessa dimensione

Primo, useremo Numba come Jit (just in time) senza una vettorizzazione esplicita.
Eviteremo un grande uso di memoria avremo un migliore uso di località: 

In [26]:
@jit()
def proc_numba(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [27]:
%timeit proc_numba(x,y)

21.4 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


notiamo subito un incremento di velocità, proviamo ora la versione vettorizzata

In [28]:
@vectorize
def vec_numba(x,y):
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    return z * ( z - .88 ) 

In [29]:
%timeit vec_numba(x,y)

6.55 µs ± 389 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


vediamo che la velocità è incrementata considerevolmente anche rispetto al codice numpy.

Utilizziamo ora numba per fare i calcoli delle features polinomiali


In [30]:
@jit(nopython=True)
def vec_poly(x, res):
    m,n=x.shape
    feat_idx=0
    for i in range(n):
        v1=x[:,i]
        for k in range(m): res[k,feat_idx] = v1[k]
        feat_idx+=1
        for j in range(i,n):
            for k in range(m): res[k,feat_idx] = v1[k]*x[k,j]
            feat_idx+=1

### Row-Major e Column-Major

Prendiamo spunto dall'[articolo di Eli Bendersky](https://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays/)
"Il sistema row-major di una matrice mette le righe in regione di memoria contigue, il sistema column-major fa lo stesso con le colonne. Conoscere il layout è importante per avere buone performance non c'è nessuna risposta su quale sia il layout migliore in generale."


* **Column-major layout:** Fortran, Matlab, R, and Julia
* **Row-major layout:** C, C++, Python, Pascal, Mathematica



In [31]:
trn = np.asfortranarray(trn)
test = np.asfortranarray(test)

In [32]:
m,n=trn.shape
n_feat = n*(n+1)//2 + n
trn_feat = np.zeros((m,n_feat), order='F')
test_feat = np.zeros((len(y_test), n_feat), order='F')

In [33]:
vec_poly(trn, trn_feat)
vec_poly(test, test_feat)

In [34]:
regr.fit(trn_feat, y_trn)

LinearRegression()

In [35]:
regr_metrics(y_test, regr.predict(test_feat))

(58.23909057980775, 45.12305980964734)

In [36]:
%timeit vec_poly(trn, trn_feat)

23.2 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
%timeit poly.fit_transform(trn)

322 µs ± 31.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
322/23.2

13.879310344827587

## Regolarizzazione e rumore

La regolarizzazione è un modo per ridurre l'over-fitting e creare modelli che generalizzano bene sui nuovi dati.

### Regolarizzazione

La regressione Lasso usa una regolarizzazione con la penalità L1 che favorisce la generazione di coefficienti sparsi. Il paramentro $\alpha$ è usato per gestire quanto penalizzare Scikit Learn esegue una cross validation con differenti valori $\alpha$ 

Se vogliamo maggiori dettagli rimando a [Coursera](https://www.coursera.org/lecture/machine-learning-data-analysis/what-is-lasso-regression-0KIy7).

In [41]:
reg_regr = linear_model.LassoCV(n_alphas=100)

In [42]:
reg_regr.fit(trn_feat, y_trn)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2764.5887538234238, tolerance: 180.36886241134752
  model = cd_fast.enet_coordinate_descent_gram(


LassoCV()

In [43]:
reg_regr.alpha_

0.00917592564562634

In [44]:
regr_metrics(y_test, reg_regr.predict(test_feat))

(49.14163972811118, 38.80085098513197)

In [46]:
hregr = linear_model.HuberRegressor()
hregr.fit(trn, y_trn)
regr_metrics(y_test, hregr.predict(test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


(50.58141515960356, 40.09214129926889)